In [ ]:
from utils.OBJ_helper import OBJ
import os
import h5py
import numpy as np
import trimesh
import trame
from utils.pickel_io import load_from_memory
from utils.Blendshape import DeformationComponents
from scipy import sparse
from scipy import linalg
from scipy.sparse.linalg import splu
from utils.Geodesic_dist import compute_topological_laplacian
from utils.vis_tools import VisPointsAttributes
path_to_sample = os.path.join(os.getcwd(), "samples", "3dgs")

In [ ]:
from datetime import datetime
# get current date and year
now = datetime.now()

date = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y")
print(date)
time = now.strftime("%H_%M")
print("time:", time)

# select dataset

In [ ]:
dataset = "trained_3dgs"
# dataset = "tracked_mesh"

if dataset == "tracked_mesh":
    path_to_dataset = os.path.join(os.getcwd(), "samples", "deformation_components", dataset)
elif dataset == "trained_3dgs":
    path_to_dataset = os.path.join(os.getcwd(), "samples", "deformation_components", dataset)

# Sparse localized deformation components
- input: deformation matrix (zero-mean) 
$$\mathbf{X} \text{ (shape = [\#trackedMeshes, \#Vertices])}$$

- output: sparse localized deformation component (shape = [#components, #vertices])
$$\mathbf{C} \text{ (shape = [\#Components, \#Vertices])}$$
    This is from the matrix factorization inducing sparsity in matrix $C$

In [ ]:
import os
import numpy as np
import trimesh

from utils.OBJ_helper import OBJ
from utils.Blendshape import FaceMask
from utils.Blendshape import ZeroMeanDefMatrix
from utils.Geodesic_dist import compute_topological_laplacian
from utils.vis_tools import VisPointsAttributes
from utils.pickel_io import dump_pckl, load_from_memory
from utils.Geodesic_dist import GeodesicDistHeatMethod, GeodesicDistSimple, compute_support_map, compute_support_map_gauss
from utils.converter import vector2MatNx3
from utils.common_utils import project_weight, proxy_l1l2

from scipy import sparse
from scipy import linalg
from scipy.sparse.linalg import splu

# use efficient implementation of sparse Cholesky factorization.
from sksparse.cholmod import cholesky_AAt, cholesky


In [ ]:
# get identical data matrix from PCA and MBSPCA

# tracked mesh
if dataset == "tracked_mesh":
    pca_hdf5 = "tracked_mesh_5perExp_trimeshPCA_dcs.hdf5"
    pca_f = h5py.File(os.path.join(path_to_dataset, pca_hdf5), 'r')
elif dataset == "trained_3dgs":
    pca_hdf5 = "Final_3dgs_87652_xyz_PCAMBSPCA_5perExp_trimesh_dcs.hdf5"
    pca_f = h5py.File(os.path.join(path_to_dataset, pca_hdf5), 'r')


In [ ]:
dataMat = np.asarray(pca_f['dataMat'])
dataMat = dataMat.reshape((dataMat.shape[0], -1, 13))
print(f"Data matrix: {dataMat.shape}")
facemask = np.asarray(pca_f["faceMask"])
facemask = facemask.reshape((-1, 3))
print(f"Face mask: {facemask.shape}")

# MEAN: original vertex list of neutral face mesh
# get the first frame of neutral face expression in the trained 3dgs
MEAN = dataMat[0]
print(f"MEAN: {MEAN.shape}")

Gaussian_MEAN = np.asarray(pca_f['MEAN'])
print(f"Gaussian_MEAN: {Gaussian_MEAN.shape}")

Gaussian_STD = np.asarray(pca_f["STD"])
print(f"Gaussian STD: {Gaussian_STD.shape}")

# tris: triangle list (index tuples for triangle mesh)
tris = np.asarray(pca_f["tris"])
print(f"triangle list: {tris.shape}")

# get the number of verts
Nverts = int(MEAN.shape[0])

create a masked centralizedd matrix

In [ ]:
# centralized data matrix
N_cent_X = dataMat
print(N_cent_X.shape)

select mesh loader

In [ ]:
mesh_loader = "trimesh"

In [ ]:
print(f"shape of data matrix: {N_cent_X.shape}")
print(f"shape of triangle list: {tris.shape}")
print(f"Number of vertices: {Nverts}")
print(f"Mesh loader: {mesh_loader}")

Visualize mean mesh

In [ ]:
# from utils.converter import vector2MatNx3
# _MEAN_MatNx3 = vector2MatNx3(MEAN.flatten(), Nverts)
# VisPointsAttributes(_MEAN_MatNx3, None, cmap = 'jet')

# Support Region Computation

Obtain triangle list
- Since the tracked meshes are topologically equivalent, we can get triangle list in advance from a sample.obj

load obj file using trimesh loader

In [ ]:
# reference verts and tris from the sample object file
mesh = trimesh.load(os.path.join(path_to_sample, "sample_face.ply"), force='mesh')
list_vertices = mesh.vertices
list_triangles = mesh.faces
verts = np.asarray(list_vertices)
tris = np.asarray(list_triangles)

# reference verts and tris from the first frame of the neutral expression
# verts = dataMat[0]
# tris = tris
print(verts.shape)
print(tris.shape)
CENTER = 2658

In [ ]:
MEAN_5509 = verts
TRIS_5509 = tris

In [ ]:
# verts: (N, 3) array (float)
# tris: (m, 3) array (int): indices into the verts array
print(tris.shape)
print(verts.shape)

Triangle list conversion
- index should be start from 0 to #num_vertex

In [ ]:
if tris.min() > 0:
    for triangle in tris:
        for i in range(3):
            # print(triangle)
            triangle[i] = triangle[i] - int(1)

Obtain distance function

In [ ]:
# heat method
gdd = GeodesicDistHeatMethod(verts, tris)
phi = gdd(CENTER) #the vertex on top of a nose
# visualize support map
# gdd.visualize_distance_func()

# simple method
# simple_gdd = GeodesicDistSimple(verts=verts, tris=tris)
# phi_simple = simple_gdd(CENTER)
# simple_gdd.visualize_distance_func()


In [ ]:
def attribute_subd2(attribute, sample_mesh):
    """
    sample_mesh: should be mesh in trimesh data structure (5509 verts)
    attribute: attribute of each vertices (attribute should be in shape [5509, #attribs])
    """
    subd1_vertices, subd1_faces = trimesh.remesh.subdivide(
        vertices=np.hstack((sample_mesh.vertices, attribute)),
        faces = sample_mesh.faces
        )

    subd1_verts = subd1_vertices[:, :3]
    subd1_phiheat = subd1_vertices[:, 3:]

    subd2_vertices, subd2_faces = trimesh.remesh.subdivide(
        vertices=np.hstack((subd1_verts, subd1_phiheat)),
        faces = subd1_faces
    )

    final_phi = subd2_vertices[:, 3:]
    final_verts = subd2_vertices[:, :3]
    return final_phi, final_verts

In [ ]:
phi_heat, final_verts = attribute_subd2(attribute=phi.reshape(phi.shape[0], -1), sample_mesh=mesh)

Generate support map (coefficient assignment)

In [ ]:
# nornalized distance function
Nphi_heat = phi_heat / max(phi_heat)
min_dist = 0.01
max_dist = 0.35
support_map = compute_support_map(Nphi_heat, min_dist, max_dist)

# Visualization of sparsity assignment strategy (T-SLDC vs. Ours)
- We use dummy distance data as sample to see each assignment

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

#Parameters to set
mu_x = 0
variance_x = 10

mu_y = 0
variance_y = 10

#Create grid and multivariate normal
x = np.linspace(-10,10,5000)
y = np.linspace(-10,10,5000)
X, Y = np.meshgrid(x,y)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X; pos[:, :, 1] = Y
rv = multivariate_normal([mu_x, mu_y], [[variance_x, 0], [0, variance_y]])

dummy_dist = 1- (rv.pdf(pos)/np.max(rv.pdf(pos)))

print(np.max(dummy_dist))
# T-SLDC
TSLDC_dummy_assignment = compute_support_map(dummy_dist, 0.1, 0.6)
# Ours (local)
Ours_dummy_assignment = compute_support_map_gauss(phi=dummy_dist, mu = 0, sigma=1.0)

#Make a 3D plot for T-SLDC
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, TSLDC_dummy_assignment,cmap='jet',linewidth=0)
plt.xticks(color="w")  
plt.yticks(color="w")  
ax.set_zlabel(r'$\Lambda_{k}$')
# plt.axis('off')

plt.show()
# plt.savefig("Trapezoidal_SLDC")

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Ours_dummy_assignment,cmap='jet',linewidth=0)
plt.xticks(color="w")
plt.yticks(color="w")
ax.set_zlabel(r'$\Lambda_{k}$')
# plt.axis('off')
plt.show()
# plt.savefig("Ours_local")


visualize support map (source #2658)

In [ ]:
# visualize support map
VisPointsAttributes(final_verts, support_map, cmap = 'coolwarm', screenshot=True, title="supportregion_")

# Pre computation
- normalized masked/centralized vertex position into [-0.5, 0.5]

In [ ]:
# used for calculating reconstruction error w.r.t masked normalized centralized data matrix X
R = N_cent_X[:, :5509, :].copy()
dataMat_5509 = N_cent_X[:, :5509, :].copy()
print(R.shape)

In [ ]:
# number of components
Ncompos = 200

# minimum/maximum geodesic distance for support region 
srMinDist = 0.2
srMaxDist = 0.3

# number of iterations to run
num_iters_max = 20

# sparsity parameter (coeffient lambda for weight of L1 regularization term)
sparse_lambda = 2

sample_5509_mesh = trimesh.load(os.path.join(path_to_sample, "sample_face.ply"), force='mesh')
# pernalty parameter for ADMM (for multiplier)
# Choice of ρ can greatly influence practical convergence of ADMM
# TOO large: not enough emphasis on minimizing a f+z
# TOO small: not enought emphasis on feasibility (Ax+Bz = c) 
rho = 10.0

# number of iteration of ADMM
num_admm_iterations = 10

# geodesic distance computation on the mean verts
gdd = GeodesicDistHeatMethod(MEAN_5509, TRIS_5509)

# Initialization
- Use deflation algorithm
- for `k` in `num_components`
    - Initialize $\mathbf{W,C} = \mathbf{0}$ and $\mathbf{R} = \mathbf{X}$
    - Find the vertex j with the highest residual in matrix $\mathbf{R}$
    $$ j = \text{argmax}_{j} \mathbf{X} - \mathbf{WC}$$
    - Find the component $C_k$ and corresponding weights $W_{:,k}$ at each step that explain maximal variance in the data via SVD/PCA
    - Subtract each of them from the deformataion matrix $\mathbf{X}$ to compute residual $\mathbf{R}$


In [ ]:
C = []
W = []

for k in range(Ncompos):
    # find the vertex explaining the most variance across the residual matrix R
    # take a norm of residual at each vertex
    magnitude = (R**2).sum(axis = 2) #shape [FxN]

    # vertex id with the most variance (residual)
    idx = np.argmax(magnitude.sum(axis = 0))

    # Find linear component explaining the motion of this vertex
    # R: shape = [F, 3]
    _U, s, Vh = linalg.svd(R[:, idx, :].reshape(R.shape[0], -1).T, full_matrices=False)
    
    # reconstruct column of matrix W at K-th column using most variant direction
    w_k = s[0] * Vh[0, :]

    # invert weight according to their projection onto the constraint set
    # This prevent problems from having negative weights
    wk_proj = project_weight(w_k)
    wk_proj_negative = project_weight(-1*w_k)

    # W_k will be replaced by the larger variance direction (+ or -)
    if(linalg.norm(wk_proj) > linalg.norm(wk_proj_negative)):
        w_k = wk_proj
    else:
        w_k = wk_proj_negative

    # flipped support region
    phi = gdd(idx)
    phi/=max(phi)
    flippedSR = 1 - compute_support_map(phi, srMinDist, srMaxDist)

    # Solve normal equation to get C_k
    # R: shape = [F, N, 3]
    # W_K: shape = [F, 1]
    # c_k: shape = [N, 3]
    # flippedSR: shape = [N, ]
    # W_k*C_k = flippedSR*R
    # C_k = (W_k^T*W_k)^{-1} W_k^T*flippedSR*R

    c_k = (np.tensordot(w_k, R, (0, 0)) * flippedSR[:, None])/ np.inner(w_k, w_k)

    C.append(c_k)
    W.append(w_k)

    # update residual
    R = R - np.outer(w_k, c_k).reshape(R.shape)

C = np.array(C) #shape = [K, N, 3]
W = np.array(W).T #shape = [F, K]


In [ ]:
print(C.shape)
print(W.shape)

# Optimization for matrix W (coefficient matrix)
- The optimization problem w.r.t matrix W is separable due to the additional constraint
- The constraints act on the weight vector $\mathbf{W_{:, k}}$ of each component separately.
- Use the block-coordinate descent algorithm, which optimize each column successively.
- Then project the updated each column of W by projecting them onto the desired W space
$$W'_{:, k} = \text{argmin}_{\mathbf{W_{:, k}\in \mathcal{V}}} ||\mathbf{X} - \mathbf{WC}||_{F}^2 = \frac{(\mathbf{R} + W_{:, k} C_k)\cdot C_k}{C_k^TC_k}$$
$$W' = \frac{W'}{\text{max}(W')}$$
$$\text{where } \mathbf{R} = \mathbf{X} - \mathbf{WC}$$


# Optimization for matrix C (deformation matrix)
- Where we fixed matrix $\mathbf{W}$, we can optimize C using convex optimization
- Use ADMM (Alternating direction method of multipliers)
    - This can optimize matrix C with good robostness of method of multipliers (faster than dual decomposition)
    - This supports decomposition (method of multipliers does not support decomposition due to the quadoratic penalty)
- Lasso problem $\mathbf{Z}$
$$\text{argmin}_{\mathbf{C}, \mathbf{Z}} ||\mathbf{X} - \mathbf{W} \cdot \mathbf{C}||_{F}^2 + \Omega(\mathbf{Z})$$
$$\text{s.t. } \mathbf{C} - \mathbf{Z} = 0$$
- Augumented Lagragian (Lagragian of ADMM)
$$\text{argmin}_{\mathbf{C}, \mathbf{Z}} ||\mathbf{X} - \mathbf{W} \cdot \mathbf{C}||_{F}^2 + \Omega(\mathbf{Z}) + \mathbf{Y}^T(\mathbf{C}-\mathbf{Z})+ (\frac{\rho}{2})||\mathbf{C}-\mathbf{Z}||_2^2$$
$$= \text{argmin}_{\mathbf{C}, \mathbf{Z}} ||\mathbf{X} - \mathbf{W} \cdot \mathbf{C}||_{F}^2 + \Omega(\mathbf{Z}) + (\frac{\rho}{2})||\mathbf{C}-\mathbf{Z} + \mathbf{U}||_2^2$$
$$\text{where } \mathbf{U} = (\frac{1}{\rho})\mathbf{Y}$$

- The ADMM algorithm initializes $\mathbf{U}\in \real^{K \times 3N}$ to zero and then iterates the following steps.
- Dual ascent
$$C^* = \text{argmin}_C ||X-WC||_{F}^2 + \frac{\rho}{2}||\mathbf{C}-\mathbf{Z}+\mathbf{U}||_{F}^2 = (W^TW + \rho I)^{-1} (W^TX+\rho(Z-U))$$
$$Z^* = \text{argmin}_Z (\Omega(\mathbf{Z}) + \frac{\rho}{2}||\mathbf{C^*}-\mathbf{Z}+\mathbf{U}||_{F}^2) = proxy_{\rho}(0, (1-\frac{\Lambda_{i,k}}{\rho ||\mathbf{C}^* + \mathbf{U}||_2^2}))_{+}[\mathbf{C}^* + \mathbf{U}]$$
- Dual update
$$\mathbf{U}^* = \mathbf{U} + \mathbf{C}^* - \mathbf{Z}^*$$


In [ ]:
# original_sparsity = np.sum( * np.sqrt((C**2).sum(axis = 2)))
# original_error = (Nmasked_cent_X**2).sum()
original_error = (R**2).sum()
print(original_error)

In [ ]:
# global optimization
# F, N, _ = Nmasked_cent_X.shape
F, N, _ = R.shape

Lambda = np.empty((Ncompos, N)) # each row representing the scaler of l1 penalty depending on the locality
U = np.zeros_like(C)
print(U.shape)

list_reconstruction_errors = []
list_sparsity = []

for i in range(num_iters_max):
    # Update weights
    # fix weight matrix, optimize C (each row respectively: c_k)
    Rflat = R.reshape(F, N*3) #flattened residual, shape = [F, N*3]
    for k in range(C.shape[0]): # for c_k (kth row)
        c_k = C[k].ravel() #flatten into [1, N*3]
        ck_norm = np.inner(c_k, c_k)
        if ck_norm <= 1e-8: # if the component does not represent any deformation component
            W[:, k] = 0
            continue # to prevent dividing by 0
        
        #block coordinate descent update
        # get updated W[:,k]'
        Rflat += np.outer(W[:, k], c_k) 
        opt = np.dot(Rflat, c_k) / ck_norm 

        #project W onto the desired space from constraints
        W[:, k] = project_weight(opt)
        Rflat -= np.outer(W[:, k], c_k)

    # precomputing lambda for each component k (Regularization term)
    # spatially varying regularization strength (to encode locality)
    for k in range(Ncompos):
        ck = C[k] #not flatten
        # find vertex with the biggest displacement in component and computer support map around it
        # take displacement vector norm at each vertex and find index with maximum of norm
        idx = (ck**2).sum(axis = 1).argmax()
        phi = gdd(idx)
        phi/=max(phi)
        support_map = compute_support_map(phi, srMinDist, srMaxDist)

        # update L1 regularization strength according to this support map
        Lambda[k] = sparse_lambda * support_map
    
    # TODO
    # Inf or NaN check in W and C

    # update components
    Z = C.copy() # this is dual variable

    # optimize matrix C fixing W
    # prefactor linear solve in ADMM
    G = np.dot(W.T, W)
    # G[np.isfinite(G) == False] = 0
    # c = np.dot(W.T, Nmasked_cent_X.reshape(Nmasked_cent_X.shape[0], -1)) #Nmasked_cent_X.reshaped into [F, N*3]  
    c = np.dot(W.T, R.reshape(R.shape[0], -1)) #Nmasked_cent_X.reshaped into [F, N*3]  
    # compute inverse part
    # scipy
    solve_prefactored = linalg.cho_factor(G + rho * np.eye(G.shape[0]))

    # sksparse.cholmod
    # sparse_csc_c = sparse.csc_matrix(G + rho * np.eye(G.shape[0]))
    # solve_prefactored = cholesky(sparse_csc_c)

    # ADMM iterations
    # TODO
    #    - check cho_factor and cho_solve from scipy
    #    - create function for proxy of update of l1/l2 reguralization term
    # old_U = U.reshape(U.shape[0], -1)
    for admm_it in range(num_admm_iterations):
        # temp_U = U.reshape(U.shape[0], -1)
        # for i in range(temp_U.shape[0]):
        #     for j in range(temp_U.shape[1]):
        #         if not np.isfinite(temp_U[i][j]):
        #             if old_U[i][j] != 0.0:
        #                 print(f"{i}, {j}: {old_U[i][j]}")
        rhs = c + rho * (Z.reshape(c.shape) - U.reshape(c.shape))
        # rhs[np.isfinite(rhs)==False] = 0
        C = linalg.cho_solve(solve_prefactored, rhs).reshape(C.shape)
        # sparse_csc_rhs = sparse.csc_matrix(c + rho * (Z.reshape(c.shape) - U.reshape(c.shape)))
        # sparse_csc_lhs = solve_prefactored(sparse_csc_rhs)
        # C = sparse_csc_lhs.toarray().reshape(C.shape)
        Z = proxy_l1l2(Lambda, C+U, 1.0/rho)
        # old_U= U.reshape(U.shape[0], -1)
        U = U + C - Z

    # set updated components to dual Z
    C = Z

    # evaluate objective function
    # R = Nmasked_cent_X - np.tensordot(W, C, (1, 0)) # residual
    R = R - np.tensordot(W, C, (1, 0)) # residual
    if (i == 0):
        initial_sparsity = np.sum(np.sqrt((C**2).sum(axis = 2))) # L1 reguralization term 
        initial_reconst_error = np.sqrt(((dataMat_5509.reshape(dataMat_5509.shape[0], -1) - np.dot(W, C.reshape(C.shape[0], -1)))**2).mean())

    # sparsity = np.sum(np.sqrt(((C*facemask[None, :])**2).sum(axis = 2))) # L1/L2 reguralization term
    sparsity = np.mean(C==0)
    # reconstruction error: root mean squared error * 1000 for convenience
    # reconst_error = np.sqrt(((X.reshape(X.shape[0], -1) - np.dot(W, C.reshape(C.shape[0], -1)))**2).mean())/1e3
    reconst_error = np.sqrt(((dataMat_5509.reshape(dataMat_5509.shape[0], -1) - np.dot(W, C.reshape(C.shape[0], -1)))**2).mean())
    # print(f"Reconstruction error: {(reconst_error/initial_reconst_error)}")
    print(f"Reconstruction error: {(reconst_error)}")
    list_reconstruction_errors.append(reconst_error)
    list_sparsity.append(sparsity)
    print(f"Sparsity: {sparsity}")
    # e = ((reconst_error/initial_reconst_error)) + sparsity/initial_sparsity
    e = ((reconst_error)) + sparsity

    # convergence check
    print("iteration %03d, E=%f" % (i, e))



In [ ]:
# np.isfinite(C).all()
# np.isfinite(W).all()
# # solve_prefactored[0]
# np.isfinite(solve_prefactored[0]).all()
# U[np.isfinite(U)==False]
# np.isfinite(G).all()

In [ ]:
RMSE = np.sqrt(((dataMat_5509.reshape(R.shape[0], -1) - np.dot(W, (C).reshape(C.shape[0], -1)))**2).mean())
print(f"RMSE: {RMSE}")

In [ ]:
Sparsity = np.sum(np.sqrt(((C)**2).sum(axis = 2)))
print(Sparsity)

In [ ]:
print(W.shape)
print(C.reshape(C.shape[0], -1).shape)

In [ ]:
print(np.isfinite(W).all())
print(np.isfinite(C).all())
print(np.isfinite(solve_prefactored[0]).all())

In [ ]:
# for i in range(C.shape[0]):
#     # print(C[i].ravel())
# C[0].max()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(C.reshape(C.shape[0], -1).T)
plt.xlabel("Vertex index")
plt.ylabel("Deformation")

# error plot

In [ ]:
plt.plot(list_reconstruction_errors)
plt.xlabel("Iteration of global optimization")
plt.ylabel("RMSE (root mean squared error)")
plt.show

# sparsity plot

In [ ]:
plt.plot(list_sparsity)
plt.xlabel("Iteration of global optimization")
plt.ylabel("sparsity (L1/L2 norm)")
plt.show()

In [ ]:
# Soarsity check
# close to 1: Sparse, close to 0: Dense
sparsity_level = np.mean(C==0)
print(sparsity_level)

# obtain the subdivided deformation components 
- Due to the size of covariance matrix of subd2_dataMat[87652, 87652], we need to sample deformation components by subdivision of the mesh with the deformation as their attributes


In [ ]:
temp_C = C.copy()

In [ ]:
reshaped_C = temp_C[0]
print(reshaped_C.shape)
for i in range(1, temp_C.shape[0]):
    reshaped_C = np.concatenate((reshaped_C, temp_C[i]), axis = 1)
print(reshaped_C.shape)
C_subd2, _ = attribute_subd2(attribute=reshaped_C, sample_mesh = sample_5509_mesh)
print(C_subd2.shape)

In [ ]:
final_C = None
final_C = C_subd2[:, :3].reshape(1, 87652, 3)
for i in range (1, temp_C.shape[0]):
    final_C = np.concatenate((final_C, C_subd2[:, 3*i:3*(i+1)].reshape(1, 87652, 3)), axis = 0)
print(final_C.shape)

In [ ]:
print(np.isfinite(final_C).all())

In [ ]:
RMSE = np.sqrt(((dataMat.reshape(dataMat.shape[0], -1) - np.dot(W, (final_C).reshape(C.shape[0], -1)))**2).mean())
print(f"RMSE: {RMSE}")

plot of components (dcs) after 2x subdivision

In [ ]:
plt.plot(final_C.reshape(final_C.shape[0], -1).T)
plt.xlabel("Vertex index")
plt.ylabel("Deformation")

# export blenshape components


In [ ]:
hdf5_saveName = "3dgs_87652_xyz_SLDC_5perExp_trimesh_dcs.hdf5"
datasetName = "3dgs"

In [ ]:
f = h5py.File(os.path.join(path_to_dataset, hdf5_saveName), 'w')
# verts 5509
# [xyz, f_dc, rotation, scale]
# [16527, 16527, 22036, 16527]
# verts 87652
# [xyz, f_dc, rotation, scale]
# [262956, 262956, 350608, 262956]
dc_key = datasetName
list_attrName = ["xyz", "f_dc", "rotation", "scale"]
# dim_info = [262956, 262956, 350608, 262956]
# dset = f.create_dataset(name = "dim_info", data=np.asarray(dim_info))
# dset.attrs["list_attrName"] = str(list_attrName)
dset1 = f.create_dataset(name = "dataMat", data=dataMat)
dset2 = f.create_dataset(name = "faceMask", data=facemask)
dset3 = f.create_dataset(name = "MEAN", data = Gaussian_MEAN)
dset4 = f.create_dataset(name = "STD", data = Gaussian_STD)
dset5 = f.create_dataset(name = "sldc_W", data = W)
dset6 = f.create_dataset(name = "sldc_C", data = final_C)
dset9 = f.create_dataset(name = "tris", data = tris)



In [ ]:
# # check if they are actually stored in hdf5
# out_f = h5py.File(os.path.join(path_to_dataset, hdf5_saveName), 'r')
# print(list(out_f.keys()))
# out_set = out_f["dim_info"]
# print(out_f["sldc_W"].shape)
# print(out_f["sldc_C"].shape)
# print(out_set.attrs["list_attrName"])